<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_ChatGithub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Lake + OpenAI实现与GitHub代码聊天 - 编程学习利器

In [ ]:
# !pip install --upgrade deeplake
# !pip install --upgrade langchain deeplake openai tiktoken

In [ ]:
# !git clone https://github.com/chroma-core/chroma.git

In [2]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

import os,sys
import openai
from dotenv import load_dotenv, find_dotenv
# sys.path.append("../..")

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
print(find_dotenv())
_ = load_dotenv(find_dotenv())
print(os.environ["OPENAI_API_KEY"])

from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

from langchain.chains import LLMRequestsChain, LLMChain


llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo')
embedding_model = OpenAIEmbeddings(disallowed_special=())
# os.environ['ACTIVELOOP_TOKEN'] = ''


c:\Users\lenovo\Desktop\LangChainPlayGround\DeeperTutorials\.env
sk-lANo2jIeCWQt94UCCf5d16B7C32744279bF98b06C822D519


In [3]:
import os
from langchain.document_loaders import TextLoader

root_dir = './chroma'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass
print(len(docs))

675


In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0
    )
texts = text_splitter.split_documents(docs)
len(texts)

Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1540, which is longer than the specified 1000
Created a chunk of size 2945, which is longer than the specified 1000
Created a chunk of size 1421, which is longer than the specified 1000
Created a chunk of size 1233, which is longer than the specified 1000
Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1730, which is longer than the specified 1000
Created a chunk of size 1887, which is longer than the specified 1000
Created a chunk of size 1245, which is longer than the specified 1000
Created a chunk of size 2383, which is longer than the specified 1000
Created a chunk of size 1537, which is longer than the specified 1000
Created a chunk of size 1474, which is longer than the specified 1000
Created a chunk of s

In [15]:
from langchain.vectorstores import DeepLake
username = "wangdingiie"

# init the input vector db
db = DeepLake(dataset_path=f"hub://{username}/chroma_source", 
              embedding_function=embedding_model,
              reset=True,)
db.add_documents(texts)

# load the vector db
db = DeepLake(dataset_path="hub://wangdingiie/chroma_source", 
              read_only=True, 
              embedding_function=embedding_model
              )

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://wangdingiie/chroma_source already exists, loading from the storage


Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Dataset(path='hub://wangdingiie/chroma_source', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
 embedding  embedding  (3336, 1536)  float32   None   
    id        text      (3336, 1)      str     None   
 metadata     json      (3336, 1)      str     None   
   text       text      (3336, 1)      str     None   
Deep Lake Dataset in hub://wyang14/chroma_source already exists, loading from the storage


In [16]:


retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-3.5-turbo')
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [18]:
questions = [
    "What does Chroma do?",
    "How to use Chroma?"
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"Question:\n {question} \n")
    print(f"Answer:\n {result['answer']} \n\n")

Question:
 What does Chroma do? 

Answer:
 Chroma is an open-source embedding database that allows you to build LLM (Language Learning Model) apps by making knowledge, facts, and skills pluggable. It provides a JavaScript/TypeScript interface to communicate with a backend Chroma DB over REST. Chroma allows you to store embeddings and perform nearest neighbor searches, making it easy to work with vector databases. It also supports various embedding models such as Sentence Transformers, OpenAI embeddings, Cohere embeddings, or your own custom embeddings. 


Question:
 How to use Chroma? 

Answer:
 To utilize Chroma, you can follow these steps:

1. Install the Chroma package: You can install the Chroma package for your preferred programming language (Python or JavaScript) using package managers like pip or npm.

2. Start Chroma: Chroma needs to be running in order for the client to communicate with it. You can follow the instructions in the [Chroma Usage Guide](https://docs.trychroma.com/

In [19]:
def ask(question, chat_history):
  response = qa({"question": question, "chat_history": chat_history})
  print(f"Question:\n {question}\n")
  print(f"Answer:\n {response['answer']}\n")

In [20]:
ask("What's the main programming language used in Chroma?", chat_history)

Question:
 What's the main programming language used in Chroma?

Answer:
 The main programming language used in Chroma is Python.



In [21]:
ask('Summarize the storage part of Chroma', chat_history)

Question:
 Summarize the storage part of Chroma

Answer:
 The storage component of Chroma is a persistent backend that stores the embeddings and metadata associated with your data. It can save your database and indexes to a directory on your local machine or to a remote server. Chroma supports different storage implementations, such as DuckDB with Parquet or a REST API server.



In [22]:
ask('Tell me more about Sentence Transformers', chat_history)

Question:
 Tell me more about Sentence Transformers

Answer:
 Sentence Transformers is a Python library that provides pre-trained models for generating sentence embeddings. These embeddings are numerical representations of sentences that capture their meaning and semantic similarity. Sentence Transformers make it easier to compare and search for similar sentences in large text collections.

The library offers a variety of pre-trained models that can be used to generate sentence embeddings. These models are trained on large datasets and can encode sentences into high-dimensional vectors. The embeddings can then be used for various natural language processing tasks such as clustering, classification, semantic search, and more.

Sentence Transformers are widely used in applications like information retrieval, question-answering systems, chatbots, and text summarization. They are designed to capture the semantic meaning of sentences and enable efficient similarity search.

Chroma, the proj

In [23]:
ask('Show me some example code on how to use Chroma to store embeddings', chat_history)

Question:
 Show me some example code on how to use Chroma to store embeddings

Answer:
 Sure! Here's an example code on how to use Chroma to store embeddings:

```python
import chromadb

# Create a client
client = chromadb.Client()

# Create a collection
collection = client.create_collection(name="my_collection")

# Add documents with embeddings to the collection
documents = ["doc1", "doc2", "doc3"]
embeddings = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
collection.add(documents, embeddings)

# Query the collection
results = collection.query(query_embeddings=[[1, 2, 3]], n_results=1)
print(results)
```

In this example, we first create a client and then create a collection called "my_collection". We then add documents (with their corresponding embeddings) to the collection using the `add` method. Finally, we query the collection using a query embedding and retrieve the nearest matching document.

Remember to install the Chroma library (`pip install chromadb`) and have a running instance of Chro

In [24]:
ask('What is the Python class for Chroma query interface?', chat_history)

Question:
 What is the Python class for Chroma query interface?

Answer:
 The Python class used for the Chroma query interface is `chromadb.Client`.



In [25]:
ask('Show me the public functions of class Client', chat_history)

Question:
 Show me the public functions of class Client

Answer:
 The public functions of the Client class are:

- reset(): Resets the client.
- version(): Retrieves the version of the client.
- heartbeat(): Retrieves the heartbeat of the client.
- persist(): Throws an error as it is not implemented in the JS client.
- createCollection(name: string, metadata?: object, embeddingFunction?: CallableFunction): Creates a new collection with the specified name, metadata, and embedding function. Returns the newly created Collection object.



In [26]:
ask('What are the underlying databases used by Chroma?', chat_history)

Question:
 What are the underlying databases used by Chroma?

Answer:
 Chroma can use different databases as its backend. By default, Chroma uses DuckDB as its in-memory database. However, it also supports using DuckDB with Parquet for persistence. Additionally, Chroma can be configured to use ClickHouse as its backend for large-scale deployments.



In [27]:
ask('Which class implements the DuckDB support?', chat_history)

Question:
 Which class implements the DuckDB support?

Answer:
 The class that implements DuckDB support in Chroma is `DuckDB`.

